In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
from rasterio.warp import transform
import xarray as xr
# Initialize the Earth Engine module.
ee.Initialize()

In [2]:
#########################################################################
############################ USER INPUTS ################################
#########################################################################
# PATHS

# DOMAIN
# choose the modeling domain
domain = 'BEAU'

# path to temporary folder to store tif files from gee
TIFpath = domain + '_GEE_Downloads/'

# path to directory where you want your output .tif and .asc files
dataPath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'
#########################################################################

# Download DEM and LC data from GEE

In [3]:
# # Download dem and veg data function
# def get_topoveg(domain, OUTpath):
    
#     #path to CSO domains
#     domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
#     domains = domains_resp.json()

#     '''
#     // These are the min and max corners of your domain in Lat, Long
#     // Western Wyoming
#     // Input the minimum lat, lower left corner
#     '''
#     minLat = domains[domain]['Bbox']['latmin']
#     #// Input the minimum long, lower left corner
#     minLong = domains[domain]['Bbox']['lonmin']
#     #// Input the max lat, upper right corner
#     maxLat = domains[domain]['Bbox']['latmax']
#     #// Input the max Long, upper right corner
#     maxLong = domains[domain]['Bbox']['lonmax']


#     # This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
#     sm_resolution = int(domains[domain]['cellsize'])

#     '''// Resolution for the PRISM output. This shoud change by Latitude of the domain
#     // because the PRISM product spatial resolution is 2.5 minutes, which equals 150 arc seconds.
#     // You can use this arc-second calculator to estimate the correct value for the PRISM resolution by latitude
#     // https://opendem.info/arc2meters.html
#     // This is one arc-second in meters for 43 degrees N Latitude'''
#     one_arcsecond = 22.57
#     PRISM_resolution = one_arcsecond * 150

#     '''// Define the final output projection using EPSG codes'''
#     epsg_code = domains[domain]['mod_proj']

#     my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    
#         #################### adjust for AK ##########################
#     #need to specifiy different GEE products for above 60 deg lat
#     #if maxLat>60:
    
#     # Use the GMTED 2010 elevation data for the DEM
#     GMTED = ee.Image('USGS/GMTED2010').select('be75');
#     filename = os.path.join(OUTpath, domain+'_dem.tif')
#     geemap.ee_export_image(GMTED, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

    
#     # download NLCD data for AK
#     NLCD = ee.Image('USGS/NLCD/NLCD2011_AK');
#     lc = NLCD.select('landcover');
#     sm_class = ee.Image(-9999) \
#         .where(lc.eq(11), 24) \
#         .where(lc.eq(12), 20) \
#         .where(lc.eq(21), 21) \
#         .where(lc.eq(22), 21) \
#         .where(lc.eq(23), 21) \
#         .where(lc.eq(24), 21) \
#         .where(lc.eq(31), 18) \
#         .where(lc.eq(41), 2) \
#         .where(lc.eq(42), 1) \
#         .where(lc.eq(43), 6) \
#         .where(lc.eq(51), 6) \
#         .where(lc.eq(52), 6) \
#         .where(lc.eq(71), 12) \
#         .where(lc.eq(72), 12) \
#         .where(lc.eq(73), 12) \
#         .where(lc.eq(74), 12) \
#         .where(lc.eq(81), 23) \
#         .where(lc.eq(82), 22) \
#         .where(lc.eq(90), 9) \
#         .where(lc.eq(95), 9)

#     # mask out invalid values
#     landMask = sm_classAK.select('constant').gt(0)
#     # select only valid parts of the image
#     NLCDmasked = sm_classAK.select('constant').updateMask(landMask).cast({'constant': ee.PixelType('int', 1, 24)})
    
#     filename = os.path.join(OUTpath, domain+'_veg_test.tif');
#     geemap.ee_export_image(sm_class, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

In [4]:
# Download dem and veg data function
def get_topoveg(domain, OUTpath):
    
    #path to CSO domains
    domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
    domains = domains_resp.json()

    # These are the min and max corners of your domain in Lat, Long
    minLat = domains[domain]['Bbox']['latmin']
    #// Input the minimum long, lower left corner
    minLong = domains[domain]['Bbox']['lonmin']
    #// Input the max lat, upper right corner
    maxLat = domains[domain]['Bbox']['latmax']
    #// Input the max Long, upper right corner
    maxLong = domains[domain]['Bbox']['lonmax']

    # This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
    sm_resolution = int(domains[domain]['cellsize'])

    # Define the final output projection using EPSG codes
    epsg_code = domains[domain]['mod_proj']

    my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    
        #################### adjust for AK ##########################
    #need to specifiy different GEE products for above 60 deg lat
    #if maxLat>60:
    
    # Use the GMTED 2010 elevation data for the DEM
    GMTED = ee.Image('USGS/GMTED2010').select('be75');
    filename = os.path.join(OUTpath, domain+'_dem.tif')
    geemap.ee_export_image(GMTED, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

    
    # download NLCD data for AK
    NLCD = ee.Image('USGS/NLCD/NLCD2011_AK');
    lc = NLCD.select('landcover');
    sm_classAK = ee.Image(-9999) \
        .where(lc.eq(11), 19) \
        .where(lc.eq(12), 20) \
        .where(lc.eq(21), 21) \
        .where(lc.eq(22), 21) \
        .where(lc.eq(23), 21) \
        .where(lc.eq(24), 21) \
        .where(lc.eq(31), 18) \
        .where(lc.eq(41), 2) \
        .where(lc.eq(42), 1) \
        .where(lc.eq(43), 6) \
        .where(lc.eq(51), 6) \
        .where(lc.eq(52), 6) \
        .where(lc.eq(71), 12) \
        .where(lc.eq(72), 12) \
        .where(lc.eq(73), 12) \
        .where(lc.eq(74), 12) \
        .where(lc.eq(81), 23) \
        .where(lc.eq(82), 22) \
        .where(lc.eq(90), 9) \
        .where(lc.eq(95), 9)

    # mask out invalid values
    landMask = sm_classAK.select('constant').gt(0)
    # select only valid parts of the image
    NLCDmasked = sm_classAK.select('constant').updateMask(landMask).cast({'constant': ee.PixelType('int', 1, 24)})
    
    # Use the Copernicus land cover product for the veg type
    copernicus = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V/Global')
    lc = copernicus.select('discrete_classification').toBands()
    
    sm_class = ee.Image(24) \
        .where(lc.eq(0), 18) \
        .where(lc.eq(20), 6) \
        .where(lc.eq(30), 12) \
        .where(lc.eq(40), 22) \
        .where(lc.eq(50), 21) \
        .where(lc.eq(60), 18) \
        .where(lc.eq(70), 20) \
        .where(lc.eq(80), 19) \
        .where(lc.eq(90), 9) \
        .where(lc.eq(100), 18) \
        .where(lc.eq(111), 1) \
        .where(lc.eq(112), 1) \
        .where(lc.eq(113), 2) \
        .where(lc.eq(114), 2) \
        .where(lc.eq(115), 3) \
        .where(lc.eq(116), 3) \
        .where(lc.eq(121), 4) \
        .where(lc.eq(122), 1) \
        .where(lc.eq(123), 2) \
        .where(lc.eq(124), 2) \
        .where(lc.eq(125), 3) \
        .where(lc.eq(126), 3) \
        .where(lc.eq(200), 24)
    
    #combine images
    combined_img = ee.ImageCollection([sm_class,NLCDmasked]).mosaic()
    
    
    # export lc image to tif
    filename = os.path.join(OUTpath, domain+'_veg.tif');
    geemap.ee_export_image(combined_img, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

In [5]:
# execute GEE function
get_topoveg(domain, dataPath);

Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/BEAU/BEAU_dem.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/BEAU/BEAU_veg.tif


# DEM

In [8]:
INfile = dataPath+domain+'_dem.tif'
da = xr.open_rasterio(INfile)
da

<xarray.DataArray (band: 1, y: 472, x: 1319)>
[622568 values with dtype=int16]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 2.478e+06 2.478e+06 2.476e+06 ... 2.008e+06 2.008e+06
  * x        (x) float64 -5.505e+05 -5.495e+05 ... 7.665e+05 7.675e+05
Attributes:
    transform:               (1000.0, 0.0, -551000.0, 0.0, -1000.0, 2479000.0)
    crs:                     +init=epsg:3338
    res:                     (1000.0, 1000.0)
    is_tiled:                1
    nodatavals:              (-32768.0,)
    scales:                  (1.0,)
    offsets:                 (0.0,)
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1

In [ ]:
import matplotlib
da.plot(cmap=matplotlib.cm.get_cmap("viridis", 24),vmin = 0, vmax = 24)

In [ ]:
INfile = '/nfs/attic/dfh/2020_NPRB/data/ascii/veg_BEAU.asc'
das = xr.open_rasterio(INfile)
das

In [ ]:
das.plot(cmap=matplotlib.cm.get_cmap("viridis", 24),vmin = 0, vmax = 24)

In [6]:
# dem
def DEM2SM(INfile, OUTpath):
    da = xr.open_rasterio(INfile)
    
    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"    
    
    np.savetxt(OUTpath+domain+'_dem.asc', np.squeeze(da.values), fmt='%d', header = head,comments='')

# Landcover 

### NLCD LC Codes

|Code|NLCD2016 Landclass|Code|NLCD2016 Landclass|
| :-: | :-: | :-: | :-: |
|11 | open water |51 | dwarf shrub|
|12| ice / snow |52 | shrub/scrub|
|21 | developed; open space |71 | grassland/herbaceous|
|22 | developed; low intensity|72 | hedge/herbaceous|
|23 | developed; med intensity|73 | lichens|
|24 | developed; high intensity|74 | moss|
|31 | barren; rock, sand, clay|81 | pasture/hay|
|41 | deciduous forest|82 | cultivated crops|
|42 | evergreen forest|90 | woody wetlands|
|43 | mixed shrub|95 | emergent herbaceous wetlands|


### Snowmodel LC Codes

|Code  |Landcover Class |Code  |Landcover Class |
| --- | --- | --- | --- |
|1     | coniferous forest |13    | subalpine meadow  |      
|2     | deciduous forest |14    | tundra (non-tussock) |      
|3     | mixed forest |15    | tundra (tussock) |           
|4     | scattered short-conifer |16    | prostrate shrub tundra | 
|5     | clearcut conifer |17    | arctic gram. wetland |       
|6     | mesic upland shrub |18    | bare |       
|7     | xeric upland shrub |19    | water/possibly frozen |       
|8     | playa shrubland |20    | permanent snow/glacier |         
|9     | shrub wetland/riparian |21    | residential/urban |   
|10    | erect shrub tundra |22    | tall crops |       
|11    | low shrub tundra |23    | short crops |        
|12    | grassland rangeland  |24    | ocean |    

In [7]:
# landcover data 
def LC2SM(INfile,OUTpath):
    da = xr.open_rasterio(INfile)

    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"

    np.savetxt(OUTpath+domain+'_veg.asc', np.squeeze(da.values), fmt='%d', header = head,comments='')

In [ ]:
# # landcover data 
# def LC2SM(INfile,OUTpath):
#     da = xr.open_rasterio(INfile)
#     data = np.squeeze(da.values)

#     #ascii header 
#     head = "ncols\t"+str(da.shape[2])+"\n" \
#     "nrows\t"+str(da.shape[1])+"\n" \
#     "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
#     "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
#     "cellsize\t"+str(int(da.res[0]))+"\n" \
#     "NODATA_value\t-9999"
    
#     #reassign lc from NLCD to SM classes
#     DIR=np.empty([da.shape[1],da.shape[2]])
#     DIR[data == 11 ]=19
#     DIR[data == 12 ]=20
#     DIR[data == 21 ]=21
#     DIR[data == 22 ]=21
#     DIR[data == 23 ]=21
#     DIR[data == 24 ]=21
#     DIR[data == 31 ]=18
#     DIR[data == 41 ]=2
#     DIR[data == 42 ]=1
#     DIR[data == 43 ]=6
#     DIR[data == 51 ]=6
#     DIR[data == 52 ]=6
#     DIR[data == 71 ]=12
#     DIR[data == 72 ]=12
#     DIR[data == 73 ]=12
#     DIR[data == 74 ]=12
#     DIR[data == 81 ]=23
#     DIR[data == 82 ]=22
#     DIR[data == 90 ]=9
#     DIR[data == 95 ]=9
#     DIR.astype(int)
#     np.savetxt(OUTpath+domain+'_veg.asc', DIR, fmt='%d', header = head,comments='')

# Lat long grids

In [8]:
def LTLN2SM(INfile,OUTpath):
    da = xr.open_rasterio(INfile)

    # Compute the lon/lat coordinates with rasterio.warp.transform
    ny, nx = len(da.y), len(da.x)
    x, y = np.meshgrid(da.x, da.y)

    # Rasterio works with 1D arrays
    lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                         x.flatten(), y.flatten())
    lon = np.asarray(lon).reshape((ny, nx))
    lat = np.asarray(lat).reshape((ny, nx))

    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"
    np.savetxt(OUTpath+domain+'_grid_lat.asc', lat, fmt='%2.5f', header = head,comments='')
    np.savetxt(OUTpath+domain+'_grid_lon.asc', lon, fmt='%4.5f', header = head,comments='')

# Execute functions

In [9]:
# generate topo
INfile = dataPath+domain+'_dem.tif'
DEM2SM(INfile, dataPath)
#generate veg
INfile = dataPath+domain+'_veg.tif'
LC2SM(INfile, dataPath)
#generate lat lon grids
LTLN2SM(INfile,dataPath)